In [1]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
    import tensorflow_hub as hub
    import numpy as np
    
    from elasticsearch import helpers
    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

Instructions for updating:
non-resource variables are not supported in the long term
Loaded  .. . . . . . . .


In [2]:
df = pd.read_excel("elastic_Search_Data.xlsx")
print("read")

read


In [3]:
df.head()

,filename,country,Product name,Refined Product Name,para_text,Possible Questions,Comment
0,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is indicated f...,What is ATACAND used for\nWhat is ATACAND used...,NaN
1,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Geriatrics (> 65 years of age) : No overall di...,What is the efficacy of ATACAND in elderly pop...,NaN
2,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Pediatrics (6 to 17 years of age): \n•\tHypert...,What is ATACAND used for in children\nWhat is ...,NaN
3,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is contraindic...,Who is ATACAND conracindicated in\nWho is ATAC...,NaN
4,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),There is evidence that co-administration of an...,Can ATACAND be co-administered with aliskiren\...,NaN


In [4]:
graph = tf.Graph()
with tf.Session(graph = graph) as session:
    print("Downloading pre-trained embeddings from tensorflow hub…")
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    text_ph = tf.placeholder(tf.string)
    embeddings = embed(text_ph)
    print("Done.")
    print("Creating tensorflow session…")
    session = tf.Session()
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    print("Done.")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Done.
Creating tensorflow session…
Done.


In [5]:
def embed_text(text):
    text=[text]
    vectors = session.run(embeddings, feed_dict={text_ph: text})
    temp= [vector.tolist() for vector in vectors]
    t = temp[0][:-15]
    return t

In [6]:
df["ml_vector"] = df["para_text"].apply(embed_text)

In [7]:
df.head()

,filename,country,Product name,Refined Product Name,para_text,Possible Questions,Comment,ml_vector
0,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is indicated f...,What is ATACAND used for\nWhat is ATACAND used...,NaN,"[0.07093886286020279, 0.028983402997255325, 0...."
1,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Geriatrics (> 65 years of age) : No overall di...,What is the efficacy of ATACAND in elderly pop...,NaN,"[-0.018367597833275795, 0.020013825967907906, ..."
2,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Pediatrics (6 to 17 years of age): \n•\tHypert...,What is ATACAND used for in children\nWhat is ...,NaN,"[0.02739354595541954, 0.020660296082496643, 0...."
3,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is contraindic...,Who is ATACAND conracindicated in\nWho is ATAC...,NaN,"[-0.028609000146389008, -0.05069785192608833, ..."
4,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),There is evidence that co-administration of an...,Can ATACAND be co-administered with aliskiren\...,NaN,"[0.04712942987680435, 0.03343551233410835, 0.0..."


In [8]:
len(df["ml_vector"][0])

497

In [9]:
df["ml_vector"][0]

[0.07093886286020279,
 0.028983402997255325,
 0.01460401713848114,
 0.029810290783643723,
 0.01314940769225359,
 -0.038807179778814316,
 -0.01850508339703083,
 0.00223507615737617,
 -0.03753262013196945,
 0.004319949075579643,
 -0.053194545209407806,
 0.0021864220034331083,
 -0.011297975666821003,
 0.06413368135690689,
 -0.009912106208503246,
 0.01757441833615303,
 -0.046726614236831665,
 -0.004263949114829302,
 0.053178027272224426,
 -0.05379520729184151,
 0.027088100090622902,
 -0.039746664464473724,
 -0.05413179472088814,
 0.07871229201555252,
 0.07275443524122238,
 -0.02662213332951069,
 -0.04835599660873413,
 -0.01772466115653515,
 -0.05842921882867813,
 -0.07537684589624405,
 0.037248946726322174,
 -0.02896687760949135,
 -0.033634766936302185,
 -0.06597630679607391,
 0.07462296634912491,
 0.02698737569153309,
 -0.08006026595830917,
 -0.027396131306886673,
 0.014733620919287205,
 -0.04150641709566116,
 0.006558993365615606,
 0.010348539799451828,
 -0.02121901512145996,
 0.03167850

In [10]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "ml_vector":{
         "type":"dense_vector",
         "dims":497
      } 
    }
   }
}

In [11]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

python client created


In [16]:
#creating the database
IndexName = 'thisisit'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [17]:
my

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'thisisit'}

In [18]:
df.columns

Index(['filename', 'country', 'Product name', 'Refined Product Name',
       'para_text', 'Possible Questions', 'Comment', 'ml_vector'],
      dtype='object')

In [19]:
df22 = df.to_dict('records')
df22

[{'filename': 'Candesartan_PM_Reviewed.docx',
  'country': 'Canada',
  'Product name': 'Candesartan cilexetil',
  'Refined Product Name': 'Atacand (Candesartan Cilexetil)',
  'para_text': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'Possible Questions': 'What is ATACAND used for\nWhat is ATACAND used for in Canada\nHow can ATACAND be used\nHow can ATACAND be used in Canada\nCan ATACAND be used with calcium channel blockers\nCan ATACAND be used with calcium channel blockers in Canada\nCan ATACAND (candesartan cilexetil) be indicated for heartfailure',
  

In [20]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("para_text",""),
    'ml_vector':line.get('ml_vector', "")})


In [21]:
arr[0]

{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'ml_vector': [0.07093886286020279,
  0.028983402997255325,
  0.01460401713848114,
  0.029810290783643723,
  0.01314940769225359,
  -0.038807179778814316,
  -0.01850508339703083,
  0.00223507615737617,
  -0.03753262013196945,
  0.004319949075579643,
  -0.053194545209407806,
  0.0021864220034331083,
  -0.011297975666821003,
  0.06413368135690689,
  -0.009912106208503246,
  0.01757441833615303,
  -0.046726614236831665,
  -0.004263949114829302,
  0.053178027272224426,
  -0.05379520729184151,
  0.02708810

In [22]:
print("this is new")
resp=es.index(index=IndexName,doc_type="_doc",body=arr[0])


this is new


In [23]:
#inserting into the database
#doc_type similar to table in rdbms

for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [24]:
resp

{'_index': 'thisisit',
 '_type': '_doc',
 '_id': '98',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 99,
 '_primary_term': 1}

In [25]:
#testing
que = "what is ATACAND?"
que_vec=embed_text(que)



In [26]:
que_vec

[0.04098149389028549,
 -1.837481431721244e-05,
 -0.01898498833179474,
 0.08209481090307236,
 -0.0534026138484478,
 0.09196053445339203,
 0.0028756267856806517,
 -0.03135484457015991,
 -0.03886687010526657,
 0.018481288105249405,
 -0.020367711782455444,
 -0.06439442187547684,
 -0.02234770357608795,
 0.0022012286353856325,
 0.02685379609465599,
 0.10510620474815369,
 0.006513011641800404,
 0.005983566865324974,
 0.07858031988143921,
 -0.03173983097076416,
 -0.046275898814201355,
 0.053890541195869446,
 -0.11198372393846512,
 -0.05182747542858124,
 -0.003643444739282131,
 -0.027352310717105865,
 -0.0665646344423294,
 -0.08463604748249054,
 -0.01895020715892315,
 0.0040874602273106575,
 -0.014260070398449898,
 -0.09420763701200485,
 -0.06633937358856201,
 -0.01766885630786419,
 0.04343526065349579,
 0.019089818000793457,
 0.037959810346364975,
 -0.05538804829120636,
 -0.015445102006196976,
 0.06644381582736969,
 0.03723593428730965,
 0.011389476247131824,
 0.05316983908414841,
 0.002138760

In [36]:

script_query = {
"script_score": {
"query": {"match_all":{}},
"script": {
"source": "cosineSimilarity(params.query_vector, doc['ml_vector']) ",
"params": {"query_vector": que_vec }
}}}


In [37]:
es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

C:\Users\srini\anaconda3\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


{'took': 394,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 100, 'relation': 'eq'},
  'max_score': 0.3438513,
  'hits': [{'_index': 'thisisit',
    '_type': '_doc',
    '_id': '63',
    '_score': 0.3438513,
    '_source': {'answer': 'The dosage of ATACAND (candesartan cilexetil) must be individualized .'}},
   {'_index': 'thisisit',
    '_type': '_doc',
    '_id': '93',
    '_score': 0.24572347,
    '_source': {'answer': 'Metabolism : Candesartan cilexetil is rapidly and completely bioactivated to candesartan by ester hydrolysis during absorption from the gastrointestinal tract. It undergoes minor hepatic metabolism by O-deethylation to an inactive metabolite. In vitro studies indicate that cytochrome P450 isoenzyme CYP 2C9 is involved in the biotransformation of candesartan to its inactive metabolite. Based on in vitro data, no interaction would be expected to occur in vivo with drugs whose metabolism is depende

In [33]:
es.get(index=IndexName,doc_type="_doc",id=3)

{'_index': 'thisisit',
 '_type': '_doc',
 '_id': '3',
 '_version': 1,
 '_seq_no': 4,
 '_primary_term': 1,
 'found': True,
 '_source': {'answer': 'ATACAND (candesartan cilexetil) is contraindicated in: \n\n•\tPatients who are hypersensitive to this drug or to any ingredient in the formulation or component of the container. For a complete listing, see DOSAGE FORMS, COMPOSITION AND PACKAGING section of the product monograph.\n\n•\tChildren aged <1 year.\n•\tPregnant women (see WARNINGS AND PRECAUTIONS, Special Populations, Pregnant Women).\n\n•\tNursing women (see WARNINGS AND PRECAUTIONS, Special Populations, Nursing Women).\n\n•\tCombination with aliskiren-containing drugs in patients with diabetes mellitus (type 1 or type 2) or moderate to severe renal impairment (GFR < 60 mL/min/1.73m2) (see WARNINGS and PRECAUTIONS, Dual Blockade of the Renin\n\n•\t-Angiotensin System (RAS) and Renal, and DRUG INTERACTIONS, Dual Blockade of the Renin- Angiotensin-System (RAS) with ARBs, ACEIs or alis

In [ ]:
Query = {
   "_source":[
      "title"
   ],
   "size":10,
   "query":{
      "script_score":{
         "query":{
            "match":{
               "title":que
            }
         },
         "script":{
            "source":"cosineSimilarity(params.query_vector, 'ml_vector') + 1.0",
            "params":{
               "query_vector":que_vec
            }
         }
      }
   }
}

In [ ]:
Query

In [ ]:
res=es.indices.